### Scikit-learn API training #2

* __train_test_split( )__  
  
    X_train, X_test, y_train, y_test = train_test_split(feature_data, target_data,...)
    - test_size: 전체 데이터에서 테스트 데이터 크기 결정 (default: 0.25)
    - train_size: 전체 데이터에서 학습용 데이터 크기 결정
    - shuffle: 데이터를 분리하기 전에 데이터를 미리 섞을지 결정 (default: True)
    - random_state: 함수를 호출할 때마다 동일한 학습/테스트용 데이터 세트를 생성하기 위해 주어지는 난수 값.


#### Cross Validation  
  
* training dataset에서 validation dataset을 따로 분리함
  
* __K fold cross validation__  

  __[작동원리]__
  
  - K = 5인 경우 전체 train_data를 4개의 학습 데이터, 1개의 검증 데이터로 나누어 모델 학습/검증을 시행
    
  - 그러면 총 5번의 검증 평가가 이루어짐  
      평가1: 1 to 4(train),5(valid), 평가2: 2 to 5(train),1(valid), 평가3: 3,4,5,1(train), 2(valid) .....
  - 교차 검증 최종 평가는 평가1 부터 평가 5까지의 결과의 평균으로 계산
  
  __[종류]__
  
  - __K fold__ : 일반적인 위의 방법의 K 폴드
  
  - __Stratified K fold__ : 불균형한(imbalanced) 분포도를 가진 레이블(결정 클래스) 데이터 집합을 위한 K 폴드 방식. 학습 데이터와 검증 데이터 세트가 갖는 레이블 분포도가 유사하도록 검증 데이터 추출
  
  __[프로세스]__
  
  - 1. 폴드 세트 설정 (K = ?)
  
  - 2. for loop에서 반복적으로 학습/검증 데이터 추출 및 학습과 예측 수행
  
  - 3. 폴드 세트별로 예측 성능의 평균을 최종 성능 평가 지표로 사용  
    
    
* __cross_val_score( )__ : 한꺼번에 위의 CV 프로세스를 실행 가능한 함수  
  
* __GridSearchCV( )__ : 교차 검증 및 최적 하이퍼 파라미터 튜닝을 한번에 실행 가능한 함수

In [1]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
import numpy as np

iris = load_iris() # data load
features = iris.data
label = iris.target

DT_df = DecisionTreeClassifier(random_state = 111) # model load

# generate K fold instance
kfold = KFold(n_splits = 5) # K = 5 (default = 3)
cross_val_accuracy = []

* __K fold__

In [2]:
N = 0
# KFold의 .split()을 이용하여 행 index를 array로 반환
for train_index, test_index in kfold.split(features): # K fold loop
    # KFold index를 가지고 실제 data의 해당 index 값을 반환
    X_train,X_test = features[train_index], features[test_index]
    y_train,y_test = label[train_index], label[test_index]
    
    # 학습 및 예측
    DT_df.fit(X_train,y_train)
    pred = DT_df.predict(X_test)
    N += 1
    
    # 반복마다 accuracy 측정
    accuracy = np.round(accuracy_score(y_test,pred),4)
    train_size = X_train.shape[0]
    test_size = X_test.shape[0]
    
    # 반복마다 결과 출력
    print('\n #{0} cross_validation accuracy: {1}, train_data_size: {2}, valid_data_size: {3}'
          .format(N,accuracy,train_size,test_size))
    print('#{0} validation set index: {1}'.format(N,test_index))
    
    cross_val_accuracy.append(accuracy) # 해당 K 마다 계산된 accuracy 저장
    
print('\n ### Mean validation accuracy: ',np.mean(cross_val_accuracy)) # 최종 CV 결과


 #1 cross_validation accuracy: 1.0, train_data_size: 120, valid_data_size: 30
#1 validation set index: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]

 #2 cross_validation accuracy: 0.9667, train_data_size: 120, valid_data_size: 30
#2 validation set index: [30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53
 54 55 56 57 58 59]

 #3 cross_validation accuracy: 0.8667, train_data_size: 120, valid_data_size: 30
#3 validation set index: [60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83
 84 85 86 87 88 89]

 #4 cross_validation accuracy: 0.9333, train_data_size: 120, valid_data_size: 30
#4 validation set index: [ 90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119]

 #5 cross_validation accuracy: 0.7333, train_data_size: 120, valid_data_size: 30
#5 validation set index: [120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 1

* __Stratified K fold__

In [3]:
import pandas as pd

iris_df = pd.DataFrame(data = iris.data, columns = iris.feature_names)
iris_df['label'] = iris.target

'''
## K fold를 이용하기 위해선 학습과 검증 데이터 내의 label 개수가 거의 유사한지 check 해야함
'''

iris_df['label'].value_counts() # 레이블 분포도 확인

2    50
1    50
0    50
Name: label, dtype: int64

In [4]:
# Kfold에선  K = 3인 경우 전체 데이터가  150이므로 50식 쪼개면 레이블 0,1,2가 고루 학습되지 않음
#kfold = KFold(n_splits = 3)

from sklearn.model_selection import StratifiedKFold

S_kfold = StratifiedKFold(n_splits = 3)

N = 0
for train_index, test_index in S_kfold.split(iris_df,iris_df['label']): # S_Kfold에선 반드시 target값도 포함
    N += 1
    label_train = iris_df['label'].iloc[train_index]
    label_test = iris_df['label'].iloc[test_index]
    
    print('## Cross Validation: {}'.format(N))
    print('train data distribution: \n', label_train.value_counts())
    print('test data distribution: \n', label_test.value_counts())

## Cross Validation: 1
train data distribution: 
 2    33
1    33
0    33
Name: label, dtype: int64
test data distribution: 
 2    17
1    17
0    17
Name: label, dtype: int64
## Cross Validation: 2
train data distribution: 
 2    33
1    33
0    33
Name: label, dtype: int64
test data distribution: 
 2    17
1    17
0    17
Name: label, dtype: int64
## Cross Validation: 3
train data distribution: 
 2    34
1    34
0    34
Name: label, dtype: int64
test data distribution: 
 2    16
1    16
0    16
Name: label, dtype: int64


* __cross_val_score( )__

In [5]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.datasets import load_iris
import numpy as np

iris_data = load_iris()
DT = DecisionTreeClassifier(random_state = 111)

data = iris_data.data
label = iris_data.target

# 성능 지표를 정확도(accuracy), 교차검증 세트(K)는 3개로 분할
scores = cross_val_score(DT,data,label,scoring = 'accuracy',cv = 3)
print('cross_valid_accuracy: ',np.round(scores,4))
print('mean_cross_valid_accuracy: ',np.round(np.mean(scores),4))

cross_valid_accuracy:  [0.9804 0.9216 0.9792]
mean_cross_valid_accuracy:  0.9604


* __GridSearchCV( )__

In [6]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score

iris = load_iris()
X_train,X_test,y_train,y_test = train_test_split(iris.data,iris.target,
                                                 test_size = 0.2, random_state = 111)

DT = DecisionTreeClassifier() # 결정트리

# hyper parameters를 dictionary 형태로 설정
parameters = {'max_depth':[1,2,3],'min_samples_split':[2,3]}

In [7]:
import pandas as pd

# param_grid의 하이퍼 파라미터들을 3개의 학습/검증 fold로 나누어서 테스트 수행
# refit = True 는 가장 좋은 파라미터 설정으로 재학습 시키기 위함
grid_DT = GridSearchCV(DT,param_grid = parameters, cv = 3,
                       refit = True, return_train_score = True)

grid_DT.fit(X_train,y_train)

# GridSearchCV 결과는 GridSearchCV.cv_results_라는 딕셔너리로 저장.
scores_df = pd.DataFrame(grid_DT.cv_results_) # 보기 좋게 DataFrame으로 변환
scores_df[['params','mean_test_score','rank_test_score','split0_test_score',
          'split1_test_score','split2_test_score']]

C:\Users\92nor\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


,params,mean_test_score,rank_test_score,split0_test_score,split1_test_score,split2_test_score
0,"{'max_depth': 1, 'min_samples_split': 2}",0.691667,5,0.690476,0.692308,0.692308
1,"{'max_depth': 1, 'min_samples_split': 3}",0.691667,5,0.690476,0.692308,0.692308
2,"{'max_depth': 2, 'min_samples_split': 2}",0.950000,2,0.904762,0.948718,1.000000
3,"{'max_depth': 2, 'min_samples_split': 3}",0.950000,2,0.904762,0.948718,1.000000
4,"{'max_depth': 3, 'min_samples_split': 2}",0.958333,1,0.904762,0.974359,1.000000
5,"{'max_depth': 3, 'min_samples_split': 3}",0.950000,2,0.904762,0.948718,1.000000


In [8]:
grid_DT.cv_results_.keys() # 가지고 있는 key값 확인

dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_max_depth', 'param_min_samples_split', 'params', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score', 'split0_train_score', 'split1_train_score', 'split2_train_score', 'mean_train_score', 'std_train_score'])

In [9]:
print(grid_DT.best_params_)
print(grid_DT.best_score_)
print(grid_DT.best_estimator_)
est = grid_DT.best_estimator_
pred = est.predict(X_test) # 파라미터 튜닝 완료된 최적 모델을 가지고 predict 가능
print('test accuracy: {0:.4f}'.format(accuracy_score(y_test,pred)))

{'max_depth': 3, 'min_samples_split': 2}
0.9583333333333334
DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')
test accuracy: 0.9000
